In [10]:
# ✅ STEP 1: Install Required Libraries
!pip install -q gradio transformers sentencepiece torch requests bs4

# ✅ STEP 2: Import Modules
import gradio as gr
import requests
import warnings
warnings.filterwarnings("ignore")
import random
import re
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer
from bs4 import BeautifulSoup

# ✅ STEP 3: Grammar Correction via LanguageTool API
def grammar_correction(text):
    url = "https://api.languagetoolplus.com/v2/check"
    params = {
        "text": text,
        "language": "en-US"
    }
    response = requests.post(url, data=params)
    if response.status_code != 200:
        return text, []
    result = response.json()
    corrected = list(text)
    offset_correction = 0
    explanations = []
    for match in result.get("matches", []):
        replacements = match.get("replacements", [])
        if replacements:
            offset = match["offset"] + offset_correction
            length = match["length"]
            replacement = replacements[0]["value"]
            original_text = text[offset:offset+length]
            corrected[offset:offset+length] = list(replacement)
            offset_correction += len(replacement) - length
            explanations.append((original_text, replacement, match.get("message")))
    return "".join(corrected), explanations

# ✅ STEP 4: Clean Text Utility
def clean_text(text):
    text = re.sub(r'\s+([?.!,])', r"\1", text)
    text = re.sub(r'\.+', ".", text)
    return text.strip()

# ✅ STEP 5: Back Translation (English -> Hindi -> English)
model_en_hi = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-en-hi")
tokenizer_en_hi = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-hi")

model_hi_en = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-hi-en")
tokenizer_hi_en = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-hi-en")

def back_translate(text):
    encoded_hi = tokenizer_en_hi(text, return_tensors="pt", padding=True, truncation=True)
    translated_hi = model_en_hi.generate(**encoded_hi)
    hindi_text = tokenizer_en_hi.decode(translated_hi[0], skip_special_tokens=True)

    encoded_en = tokenizer_hi_en(hindi_text, return_tensors="pt", padding=True, truncation=True)
    translated_en = model_hi_en.generate(**encoded_en)
    english_back = tokenizer_hi_en.decode(translated_en[0], skip_special_tokens=True)

    return clean_text(english_back.replace("by day", "daily"))

# ✅ STEP 6: Tone/Style Transformation
def style_transformations(text):
    text_lower = text.lower()
    text_clean = clean_text(text)
    third_person = text_lower.replace("i ", "he ").replace(" my ", " his ").replace(" me ", " him ")

    styles = {
        "Official": f"It is officially noted that {text_lower}.",
        "Formal": f"According to formal communication, {text_clean}.",
        "Informal / Casual": f"Yeah, so basically, {text_lower}!.",
        "Polite": f"If you don't mind, {text_lower}. Please.",
        "Frank": f"Honestly, {text_clean}.",
        "Rude": f"Ugh, just {text_lower}. Already!.",
        "Sarcastic": f"Oh wow, like {text_lower}. Really matters. 😒.",
        "Humorous": f"Guess what? {text_lower}. And the market exploded with veggies. 😂.",
        "Motivational": f"Believe in yourself! Even {text_lower}. Is a step forward.",
        "Apologetic": f"I'm sorry, but {text_lower}. Had to happen.",
        "Aggressive": f"Enough! {text_clean.upper()}. Now!.",
        "Persuasive": f"You should really consider that {text_lower}.",
        "Sympathetic": f"I understand. {text_lower}. Wasn’t easy.",
        "Narrative": f"Yesterday, a story unfolded as he {text_clean}.",
        "Inquisitive": f"Did you know he {text_clean}?",
        "Commanding": f"{text_clean} — no questions asked.",
        "Grateful": f"I'm thankful that {text_lower} happened.",
        "Romantic": f"He {text_clean} with your love in his heart. ❤️.",
        "Professional": f"As per schedule, he {text_clean}.",
        "Poetic": f"Through bustling streets he wandered, for he {text_clean} was his quest. 🌸.",
        "Humanized": f"You know, it just so happened that {text_lower}. And that felt really natural."
    }
    return styles

# ✅ STEP 7: Real Plagiarism Check using search engine scraping
def check_plagiarism(text):
    try:
        search_url = f"https://www.google.com/search?q={requests.utils.quote(text)}"
        headers = {"User-Agent": "Mozilla/5.0"}
        resp = requests.get(search_url, headers=headers)
        soup = BeautifulSoup(resp.text, "html.parser")
        results = soup.select(".BNeawe span")
        found = any(text.lower() in result.get_text().lower() for result in results)
        return f"Plagiarism Detected: {'Yes' if found else 'No'} (based on search match)"
    except:
        return "Plagiarism Check Failed — Network or Blocking Issue"

# ✅ STEP 8: Final Integrated Function
def grammarly_ai(user_input):
    plagiarism_result = check_plagiarism(user_input)
    corrected, explanations = grammar_correction(user_input)
    paraphrased, _ = grammar_correction(back_translate(corrected))
    styles = style_transformations(paraphrased)

    styled_outputs = []
    for style_text in styles.values():
        fixed, _ = grammar_correction(style_text)
        styled_outputs.append(clean_text(fixed))

    df = pd.DataFrame(explanations, columns=["Incorrect", "Corrected", "Reason"])
    explanation_table = df.to_markdown(index=False)

    return [plagiarism_result, corrected, paraphrased, explanation_table] + styled_outputs

# ✅ STEP 9: Gradio Interface
interface = gr.Interface(
    fn=grammarly_ai,
    inputs=gr.Textbox(lines=5, placeholder="Enter your sentence or paragraph here..."),
    outputs=[
        gr.Textbox(label="🔍 Plagiarism Report"),
        gr.Textbox(label="✅ Grammar Corrected"),
        gr.Textbox(label="🔁 Paraphrased (Back Translation)"),
        gr.Textbox(label="💡 Grammar Explainability"),
        gr.Textbox(label="1️⃣ Official"),
        gr.Textbox(label="2️⃣ Formal"),
        gr.Textbox(label="3️⃣ Informal / Casual"),
        gr.Textbox(label="4️⃣ Polite"),
        gr.Textbox(label="5️⃣ Frank"),
        gr.Textbox(label="6️⃣ Rude"),
        gr.Textbox(label="7️⃣ Sarcastic"),
        gr.Textbox(label="8️⃣ Humorous"),
        gr.Textbox(label="9️⃣ Motivational"),
        gr.Textbox(label="🔟 Apologetic"),
        gr.Textbox(label="11️⃣ Aggressive"),
        gr.Textbox(label="12️⃣ Persuasive"),
        gr.Textbox(label="13️⃣ Sympathetic"),
        gr.Textbox(label="14️⃣ Narrative"),
        gr.Textbox(label="15️⃣ Inquisitive"),
        gr.Textbox(label="16️⃣ Commanding"),
        gr.Textbox(label="17️⃣ Grateful"),
        gr.Textbox(label="18️⃣ Romantic"),
        gr.Textbox(label="19️⃣ Professional"),
        gr.Textbox(label="20️⃣ Poetic"),
        gr.Textbox(label="✨ Humanized (Natural Style)")
    ],
    title="🧠 TextMorph AI — Grammar + Tone + Paraphrase + Plagiarism",
    description="Enter a sentence. Get grammar correction, paraphrased back translation, tone/style rewrites, and plagiarism score with grammar explainability."
)

# ✅ STEP 10: Launch
interface.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f8b3800da5b70f49ba.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
